# Ceneo scraper

# Biblioteki

In [13]:
import requests
from bs4 import BeautifulSoup

# Wysyłanie do serwera żądania dostępu do zasobu

In [14]:
product_id = "120498791"
url =  f"https://www.ceneo.pl/{product_id}#tab=reviews_scroll"
response = requests.get(url)

In [15]:
page_dom = BeautifulSoup(response.text, "html.parser")
print(type(page_dom))
opinions = page_dom.select("div.js_product-review")
print(opinions)
print(type(opinions))
opinion = page_dom.select_one("div.js_product-review")
print(type(opinion))
print(opinion)

<class 'bs4.BeautifulSoup'>
[<div class="user-post user-post__card js_product-review" data-entry-id="17802154">
<header class="user-post__header">
<div class="js_lazy user-post__avatar user-rank__avatar" data-bg="/Content/img/account/avatar/5.svg"></div>
</header>
<div class="user-post__body">
<div class="user-post__content">
<span class="user-post__author-name">
r...1</span>
<span class="user-post__author-recomendation">
<em class="recommended">Polecam</em>
</span>
<span class="user-post__score">
<span class="screen-reader-text">Ocena:</span>
<span class="score-container score-container--s js_score-container">
<span class="score-marker score-marker--s" style="width: 90.00%;"></span>
</span>
<span class="user-post__score-count">4,5/5</span>
<span class="user-post__published">
        Wystawiono
        <time datetime="2023-08-14 17:38:40">8 miesięcy temu, </time>
<time datetime="2023-08-08 18:39:36">po 6 dniach</time> użytkowania            </span>
</span>
<div class="user-post__text">

# Ekstrakcja składowych pojedyńczej opinii
|składowa|selektor|zmienna|
|--------|--------|-------|
|id opinii|["data-entry-id"]|opinion_id|
|autor|span.user-post__author-name|author|
|rekomendacja|span.user-post__author-recomendation > em|recomendation|
|gwiazdki|span.user-post__score-count|rating|
|treść|div.user-post__text|content|
|lista zalet|div.review-feature__title--positives ~ div.review-feature_item|pros|
|lista wad|div.review-feature__title--negatives ~ div.review-feature_item|cons|
|dla ilu przydatna|button.vote-yes > span|useful|
|dla ilu nieprzydatna|button.vote-no > span|useless|
|data wstawienia|span.user-post__published > time:nth-child(1)["datetime"]|publish_date|
|data zakupu|span.user-post__published > time:nth-child(2)["datetime"]|purchase_date|


In [16]:
opinion_id=opinion["data-entry-id"]
author=opinion.select_one("span.user-post__author-name")
recomendation=opinion.select_one("span.user-post__author-recomendation > em")
rating=opinion.select_one("span.user-post__score-count")
content=opinion.select_one("div.user-post__text")
pros=opinion.select("div.review-feature__title--positives ~ div.review-feature_item")
cons=opinion.select("div.review-feature__title--negatives ~ div.review-feature_item")
useful=opinion.select_one("button.vote-yes > span")
useless=opinion.select_one("button.vote-no > span")
publish_date=opinion.select_one("span.user-post__published > time:nth-child(1)")
purchase_date=opinion.select_one("span.user-post__published > time:nth-child(2)")